In [ ]:
#=import Pkg; 
Pkg.add(["IJulia", "CategoricalArrays", "LaTeXStrings",
        "CDMrdata",  "RDatasets",
        "CSV", "DataFrames",    
        "StatsBase", "StatsFuns",  "Distributions", 
        "Gadfly", "Plots", "StatsPlots",
        "Optim", "Turing", "MCMCChains", "DynamicPPL", "Zygote"
         ])
Pkg.update()
=#

In [ ]:
using Distributions
using DataFrames
using Gadfly
using Turing
using StatsFuns
using Plots
using StatsPlots
using LinearAlgebra
using CategoricalArrays
using MCMCChains
using CSV
using CDMrdata

In [ ]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 10
Threads.nthreads()

In [ ]:
# Adding standard font sizes for all future graphs will make them much easier to read

latex_fonts = Theme(major_label_font="CMU Serif", major_label_font_size=22pt,
                   minor_label_font="CMU Serif", minor_label_font_size=16pt,
                   key_title_font="CMU Serif", key_title_font_size=18pt,
                   key_label_font="CMU Serif", key_label_font_size=16pt)
Gadfly.push_theme(latex_fonts)


In [ ]:
# The number of samples is enough to create convergence later on, bigger is usually better but 600 is great
samples = 600
num_chains = 4

In [ ]:
#=
This is the sample of students and the exam scores. There are 536 students represented through rows
and 15 questions as columns. A value of "1" indicates a question answered correctly and a value of "0"
an incorrect answer.
=#
X = DataFrame(CSV.File("final_project_Miguel.csv"))

In [ ]:
I, J = size(X)
# Dimension here is due to the IRT model which uses "I" as the student dimension and "J" as the item/question dimension

In [ ]:
X_stacked = DataFrames.stack(X, All())
# This will convert the data to a long version so we can calculate the percentage attempts

In [ ]:
X_grp = groupby(X_stacked, :variable)
# this will group each repetition of student's answers as one student
X_total_items = combine(X_grp, :value => (col -> (sum(col) / 536.0) * 100) => :percentage_attempts)
# Now we get the average by performing avg calculations on the column that was grouped, this gets our avg as a float

In [ ]:
Gadfly.plot(X_total_items, x = :variable, y = :percentage_attempts, Geom.point,
Guide.xlabel("Item Number"), Guide.ylabel("Percentage Correct"), Guide.title("Item Performance")
)

#Using the gadfly package, I can now place each avg as the individual item performance.
#Each point represents the individual item's percentage correct

In [ ]:
Gadfly.plot(X_total_items, x = :percentage_attempts, Geom.density,
Guide.xlabel("Percentage Correct"), Guide.ylabel("Density"), Guide.title("Student Population Performance")
)
#Here's a slightly different way to visualize the above graph, use a density graph to see a continous line 

In [12]:
function calc_likelihood(θ, d, aⱼ, c) # based on the 3-parameter IRT eqn, we'll use theta as "ability", d as "difficulty", etc
    

    Pₓ = c .+ (1 .- c).* logistic.(θ * aⱼ.-d) # The dots adjacent to the operators lets us distribute the operation to each element of the array
  
    return Pₓ
end

@model function irt_3pl_model(Xᵢ, I, J)
    
    θ ~ filldist(Normal(0,1), I,1) #theta is the i-th dimension since it refers to students. This prior indicates that student ability is average
    aⱼ ~ filldist(LogNormal(0,1), 1,J)
    d ~ filldist(Normal(1,1), 1,J) #Using 1,1 for the prior indicates that the test questions were "harder" than normal
    c ~ filldist(Beta(), 1,J)

    Pₓ = calc_likelihood(θ, d, aⱼ, c) 
    if (Xᵢ != nothing)
        Xᵢ .~ Bernoulli.(Pₓ)
    else
        X ~ arraydist(Bernoulli.(Pₓ))
    end
    return Pₓ
end


In [ ]:
prior_chain = sample(irt_3pl_model(nothing, I, J), Prior(), samples); #we will sample 600 times from the above distributions
#This will be the basis prior chain 

In [ ]:
df_prior = DataFrame(prior_chain) #this will place our chain in a dataframe
df_prior = DataFrames.stack(df_prior, Not(["iteration","chain","lp"])) #will convert to long format
subset!(df_prior, :variable => ByRow(x-> !contains(x, "θ") && !contains(x, "X")))#reframes the data so that each row is applied


In [ ]:
prior_plot = Gadfly.plot(df_prior, x=:value, y=:lp, Geom.point, color=:variable, Guide.xlabel("Data"), Guide.ylabel("Log Likelihood"))
#This command plots the parameters in the prior to see if they converge. If they don't, we shouldn't proceed

In [ ]:
posterior_chain_nuts = sample(irt_3pl_model(Matrix(X), I, J), NUTS(), MCMCThreads(), samples, num_chains);
#for the posterior, we will sample from the prior distributions using NUTS algorithm!

In [ ]:
predict_chain = predict(irt_3pl_model(nothing, I, J), #plugging in the model to see how the model will perform
posterior_chain_nuts[:,:,1], include_all=false); #this will pull out the first chain as an argument 
df_predict = DataFrame(predict_chain) #Now we can store the chain into a Dataframe

In [ ]:
select!(df_predict, r"X") #this excludes the iterations and chain column so we can combine the rest of the columns accurately

In [ ]:
x_predict = combine(df_predict, All() .=> mode) #now we can see the correct answers for each item for each student in a wide format

In [ ]:
reshape(Matrix(x_predict), (I,J)) #now the array will reshape to the dimensions IxJ

In [ ]:
x_predict = DataFrame(reshape(Matrix(x_predict), (I,J)), string.("T", string.(1:J, pad = 2)))
#plugging in the reshape into a dataframe and clarifying each column as a test item/question

In [ ]:
#using similar logic in the item performance graph in ln 11, we stack the predicted data and group by variables to perform
#avg calculations, then we stack two layers into gadfly, with the first argument being the predicted and the second the actual
#item performance seen in ln 11. Notice the discrepancy between the dots, they mean that the prediction is not spot on.

X_stacked_pred = DataFrames.stack(x_predict, All())

X_grp_pred = groupby(X_stacked_pred, :variable)
X_total_items_pred = combine(X_grp_pred, :value => (col -> (sum(col) / 536.0) * 100) => :percentage_attempts)

set_default_plot_size(20cm, 12cm) 

Gadfly.plot(layer(X_total_items_pred, x = :variable, y = :percentage_attempts, Geom.point, color = ["Predicted Item Performance"]),
    layer(X_total_items, x = :variable, y = :percentage_attempts, Geom.point, color = ["Observed Item Data"]),
Guide.xlabel("Item Parameter"), Guide.ylabel("Percentage Correct"), Guide.title("Observed vs Predicted Item Performance")
)


In [ ]:
#Similar to the Student population performance graph from earlier, we compared how the predicted and observed data relate
#in a continuous line by creating two layers. Notice the gap in the 60th- 75th parameter.

Gadfly.plot(layer(X_total_items_pred, x = :percentage_attempts, Geom.density, color = ["Predicted Item Performance"]),
    layer(X_total_items, x = :percentage_attempts, Geom.density, color = ["Observed Item Data"]),
Guide.xlabel("Item Parameter"), Guide.ylabel("Percentage Correct"), Guide.title("Observed vs Predicted Item Performance")
)

In [ ]:
df_posterior_chain_nuts = DataFrame(describe(posterior_chain_nuts)[1])
df_parameter = transform(df_posterior_chain_nuts, :parameters => ByRow(x -> string(string.(x)[1])) => :parameter_type)
#We want to store the first chain as a dataframe so we can gather the rhat measurement later on for convergence

In [ ]:
#show(df_posterior_chain_nuts, allrows=true) #this will come in handy to check which parameters are converging e.g theta, d, c

In [ ]:
#since rhat values close to 0 means that the parameters have converged, we want to see how each one performed by setting the x 
#to parameters from above and y to rhat itself. hstack rearranges the plots to see them more clearly
h = Gadfly.plot(df_parameter, x = :parameter_type, y = :rhat, Geom.point,
Guide.xlabel("Type of Parameter"), Guide.ylabel("Rhat"), Guide.title("MCMC Convergence"), Stat.x_jitter(range=0.2),
)
hstack(h)

In [ ]:
df_posterior_theta_mean = df_posterior_chain_nuts[1:536,1:2] #this will retrieve the correct columns for theta mean
for i in 1:536 #this loop will iterate through each row and assign the correct students' ability posterior
    df_posterior_theta_mean[i, 1] = Symbol.(i)
end
df_posterior_theta_mean

In [ ]:
df_prior_theta_mean = DataFrame(describe(prior_chain)[1])[1:536, 1:2] #this will access the correct columns from the prior chain
for i in 1:536 #same concept as in cell before
    df_prior_theta_mean[i, 1] = Symbol.(i)
end
df_prior_theta_mean

In [ ]:
#By stacking two layers (prior and posterior estimates) we can see generally across all points if the posterior has diverged from
#the prior, whhich it should for Bayesian probability
ind_prob_plot = Gadfly.plot(
    layer(df_prior_theta_mean, x = 1:nrow(df_prior_theta_mean), y =:mean, Geom.point, color = ["Prior Estimate"]),
    layer(df_posterior_theta_mean, x = 1:nrow(df_posterior_theta_mean), y =:mean, Geom.point, color = ["Posterior Estimate"]),
    Guide.xlabel("Ability Parameter"), Guide.ylabel("Posterior Mean"), Guide.colorkey("variable"), Guide.Title("Student Ability Prior and Posterior Estimates")
)

In [ ]:
#to analyze free parameters, we used the similar retrieval of the posterior chain and prior chain, only now
#we want to correctly retrieve rows 537-581 and iterate through 45 times since there's 15 questions on the test * 3 parameters
#we exclude the ability parameter since we want to evaluate the testing parameters and how much they diverged from the prior
item_parameters_prior = DataFrame(describe(prior_chain)[1])[537:581, 1:2]
item_parameters_prior.item_num = 1:45

item_parameters_posterior = df_posterior_chain_nuts[537:581, :]
item_parameters_posterior = item_parameters_posterior[:, 1:2]
item_parameters_posterior.item_num = 1:45

#graphing
ind_prob_plot = Gadfly.plot(
    layer(item_parameters_prior, x =:item_num, y =:mean, Geom.point, color = ["Prior Estimate"]),
    layer(item_parameters_posterior, x =:item_num, y =:mean, Geom.point, color = ["Posterior Estimate"]),
    Guide.xlabel("Item Parameters"), Guide.ylabel("Posterior Mean"), Guide.colorkey("variable"), Guide.Title("Item Prior and Posterior Estimates")
)

In [ ]:
#show(DataFrame(describe(prior_chain)[1])) #to prepare for the ability vs. difficulty curve, we need to first check
#if we have the "summary statistics" chain

In [ ]:
#show(DataFrame(describe(prior_chain)[1])[537:581,:],allrows=true) #this will help us find specific examples of students
#that performed avg and students who performed well

In [ ]:
df_difficulty = df_posterior_chain_nuts[552:566, :] #this will be used as the dataframe for one layer
#of the difficulty vs. ability graph

In [ ]:
df_ability = DataFrame(df_posterior_chain_nuts[515, :]) #student who performed well
#we discovered this student from an earlier analysis of student performance, the slideshow will showcase this

In [ ]:
df_ability_avg = DataFrame(df_posterior_chain_nuts[504, :])# this is a student who performed poorly

In [ ]:
#we layered the mean columns of the avg student and the good student as well as the difficulty of each item.
#we wanted to use dots on a x-axis to display the distance between the good student and the bad.
#the student dot can only answer questions to the left, so this indicates a good measuring of difficulty
ind_prob_plot = Gadfly.plot(
    layer(df_ability, x =:mean, y =[0], Geom.point, color = ["Good Student Ability"]),
    layer(df_ability_avg, x =:mean, y =[0], Geom.point, color = ["Avg Student Ability"]),
    layer(df_difficulty, x =:mean, y =repeat([0], nrow(df_difficulty)), Geom.point, color = ["Item Difficulty"]), size = [3mm],
    Guide.xlabel("Posterior Mean"), Guide.ylabel("y"), Guide.colorkey("Parameter Type"), Guide.Title("Ability vs Difficulty")
)
hstack(ind_prob_plot)

In [ ]:
item_param = r"dⱼ|aⱼ|cⱼ"
item = r"\[1,1\]"
df_item_posterior_mean = subset(DataFrame(describe(posterior_chain_nuts)[1]), :parameters => ByRow(x->contains(string(x),item_param)))
#here we want to retrieve the mean by accessing the paramater column from chain 1 of posterior and input the parameters of IRT

df_ability_results = DataFrame(describe(posterior_chain_nuts)[1])
subset!(df_ability_results, :parameters => ByRow(x->contains(string(x), "θ")))
#We want to retain the same dimension of array but now isolate the theta or "ability" results




In [ ]:
df_item_curve = transform(df_ability_results, :mean => (x -> calc_likelihood(x, df_item_posterior_mean[1,2], df_item_posterior_mean[2,2], df_item_posterior_mean[3,2])) => :likelihood)
#the transform function lets us use our calc_likelihood function to use as an argument to see if ability results can be plugged into the model

In [ ]:
Gadfly.plot(df_item_curve, x=:mean, y=:likelihood, Geom.line, Guide.ylabel("Correctness Likelihood"), Guide.xlabel("Ability"), Guide.title("Item Characteristic Curve(1st Item)"))
#finally, by using the model's likelihood function, we can see that as ability increases, the probability of endorsing
# a correct answer diminishes. This is quite the opposite of what we were aiming to capture with IRT, so as a
# "business" decision, we decided not to recommend IRT 3-parameter for potential clients who want to analyze student and 
#item performance.